In [1]:
import requests
import numpy as np
import pandas as pd
import re
import key

pd.set_option('display.max_columns', None)

# Get Anime Data from API

In [2]:
# api key for the header
header = {'X-MAL-CLIENT-ID':key.api_key}

In [3]:
# example
anime_response = requests.get('https://api.myanimelist.net/v2/anime/ranking?ranking_type=all&limit=2&fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,num_episodes,start_season,broadcast,source,average_episode_duration,rating,studios',  headers=header)
anime_response_json = anime_response.json()
anime_response_json

{'data': [{'node': {'id': 5114,
    'title': 'Fullmetal Alchemist: Brotherhood',
    'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/1208/94745.jpg',
     'large': 'https://api-cdn.myanimelist.net/images/anime/1208/94745l.jpg'},
    'alternative_titles': {'synonyms': ['Hagane no Renkinjutsushi: Fullmetal Alchemist',
      'Fullmetal Alchemist (2009)',
      'FMA',
      'FMAB'],
     'en': 'Fullmetal Alchemist: Brotherhood',
     'ja': '鋼の錬金術師 FULLMETAL ALCHEMIST'},
    'start_date': '2009-04-05',
    'end_date': '2010-07-04',
    'synopsis': 'After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse\'s body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse\'s so

In [4]:
x = 0
anime_list = []

api_url = 'https://api.myanimelist.net/v2/anime/ranking?ranking_type=all&limit=500&fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,num_episodes,start_season,broadcast,source,average_episode_duration,rating,studios'

while x < 20000:
    anime_response_json = requests.get(api_url, headers=header).json()
    for i in range(500):
        anime_list.append(anime_response_json['data'][i])

    api_url = anime_response_json['paging']['next']
    x += 500

print('anime_list length:', len(anime_list))

anime_list length: 20000


In [5]:
df = pd.DataFrame(anime_list)
df.head()

,node,ranking
0,"{'id': 5114, 'title': 'Fullmetal Alchemist: Br...",{'rank': 1}
1,"{'id': 52034, 'title': '""Oshi no Ko""', 'main_p...",{'rank': 2}
2,"{'id': 51535, 'title': 'Shingeki no Kyojin: Th...",{'rank': 3}
3,"{'id': 9253, 'title': 'Steins;Gate', 'main_pic...",{'rank': 4}
4,"{'id': 41467, 'title': 'Bleach: Sennen Kessen-...",{'rank': 5}


In [6]:
# normalize
normalized_df = pd.json_normalize(df['node'])
normalized_df.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3149847,2003682,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00
1,52034,"""Oshi no Ko""",2023-04-12,NaN,Sixteen-year-old Ai Hoshino is a talented and ...,9.08,2.0,525,399851,131985,white,2022-06-09T13:01:38+00:00,2023-04-23T23:12:26+00:00,tv,currently_airing,"[{'id': 8, 'name': 'Drama'}, {'id': 72, 'name'...",11,manga,0,pg_13,"[{'id': 95, 'name': 'Doga Kobo'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[My Star],[Oshi No Ko],【推しの子】,2023.0,spring,wednesday,23:00
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.08,3.0,511,408423,139142,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN


# Data Cleaning for the Genres Column

In [7]:
normalized_df['genres'][0]

[{'id': 1, 'name': 'Action'},
 {'id': 2, 'name': 'Adventure'},
 {'id': 8, 'name': 'Drama'},
 {'id': 10, 'name': 'Fantasy'},
 {'id': 38, 'name': 'Military'},
 {'id': 27, 'name': 'Shounen'}]

In [8]:
# normalize the genres column
normalized_df2 = normalized_df.join(pd.json_normalize(normalized_df.pop('genres')))
normalized_df2.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,0,1,2,3,4,5,6,7,8,9,10,11
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3149847,2003682,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"{'id': 1, 'name': 'Action'}","{'id': 2, 'name': 'Adventure'}","{'id': 8, 'name': 'Drama'}","{'id': 10, 'name': 'Fantasy'}","{'id': 38, 'name': 'Military'}","{'id': 27, 'name': 'Shounen'}",None,None,None,None,None,None
1,52034,"""Oshi no Ko""",2023-04-12,NaN,Sixteen-year-old Ai Hoshino is a talented and ...,9.08,2.0,525,399851,131985,white,2022-06-09T13:01:38+00:00,2023-04-23T23:12:26+00:00,tv,currently_airing,11,manga,0,pg_13,"[{'id': 95, 'name': 'Doga Kobo'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[My Star],[Oshi No Ko],【推しの子】,2023.0,spring,wednesday,23:00,"{'id': 8, 'name': 'Drama'}","{'id': 72, 'name': 'Reincarnation'}","{'id': 42, 'name': 'Seinen'}","{'id': 75, 'name': 'Showbiz'}","{'id': 37, 'name': 'Supernatural'}",None,None,None,None,None,None,None
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.08,3.0,511,408423,139142,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"{'id': 1, 'name': 'Action'}","{'id': 8, 'name': 'Drama'}","{'id': 58, 'name': 'Gore'}","{'id': 38, 'name': 'Military'}","{'id': 27, 'name': 'Shounen'}","{'id': 76, 'name': 'Survival'}","{'id': 41, 'name': 'Suspense'}",None,None,None,None,None


In [9]:
# get rid of unnecessary characters in the genres columns
for col in range(0,12):
    normalized_df2[col] = normalized_df2[col].apply(lambda x: re.sub('[{}\'0-9:,]', '', str(x)))
    normalized_df2[col] = normalized_df2[col].apply(lambda x: x.replace('id', '').replace('name', '').replace('None', '').strip())
    
normalized_df2.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,0,1,2,3,4,5,6,7,8,9,10,11
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3149847,2003682,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,Action,Adventure,Drama,Fantasy,Military,Shounen,,,,,,
1,52034,"""Oshi no Ko""",2023-04-12,NaN,Sixteen-year-old Ai Hoshino is a talented and ...,9.08,2.0,525,399851,131985,white,2022-06-09T13:01:38+00:00,2023-04-23T23:12:26+00:00,tv,currently_airing,11,manga,0,pg_13,"[{'id': 95, 'name': 'Doga Kobo'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[My Star],[Oshi No Ko],【推しの子】,2023.0,spring,wednesday,23:00,Drama,Reincarnation,Seinen,Showbiz,Supernatural,,,,,,,
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.08,3.0,511,408423,139142,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,Action,Drama,Gore,Military,Shounen,Survival,Suspense,,,,,


In [10]:
# get integer location of the column 0
normalized_df2.columns.get_loc(0)

29

In [11]:
# combine genres
normalized_df2['genres'] = normalized_df2[normalized_df2.columns[29:]].agg(' '.join, axis=1)
normalized_df2['genres'] = normalized_df2['genres'].apply(lambda x: ', '.join([word for word in x.split() if word != '']))

In [12]:
normalized_df2['genres'][0]

'Action, Adventure, Drama, Fantasy, Military, Shounen'

In [13]:
# drop the normalized columns (0 to 11)
normalized_df2.drop([0,1,2,3,4,5,6,7,8,9,10,11], axis=1, inplace=True)
normalized_df2.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,studios,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3149847,2003682,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,"[{'id': 4, 'name': 'Bones'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S..."
1,52034,"""Oshi no Ko""",2023-04-12,NaN,Sixteen-year-old Ai Hoshino is a talented and ...,9.08,2.0,525,399851,131985,white,2022-06-09T13:01:38+00:00,2023-04-23T23:12:26+00:00,tv,currently_airing,11,manga,0,pg_13,"[{'id': 95, 'name': 'Doga Kobo'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[My Star],[Oshi No Ko],【推しの子】,2023.0,spring,wednesday,23:00,"Drama, Reincarnation, Seinen, Showbiz, Superna..."
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.08,3.0,511,408423,139142,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,"[{'id': 569, 'name': 'MAPPA'}]",https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv..."


# Do the Same for the Studios Column

In [14]:
# normalize the studios column
normalized_df3 = normalized_df2.join(pd.json_normalize(normalized_df2.pop('studios')))
normalized_df3.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,0,1,2,3,4,5,6,7,8,9
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3149847,2003682,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...","{'id': 4, 'name': 'Bones'}",None,None,None,None,None,None,None,None,None
1,52034,"""Oshi no Ko""",2023-04-12,NaN,Sixteen-year-old Ai Hoshino is a talented and ...,9.08,2.0,525,399851,131985,white,2022-06-09T13:01:38+00:00,2023-04-23T23:12:26+00:00,tv,currently_airing,11,manga,0,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[My Star],[Oshi No Ko],【推しの子】,2023.0,spring,wednesday,23:00,"Drama, Reincarnation, Seinen, Showbiz, Superna...","{'id': 95, 'name': 'Doga Kobo'}",None,None,None,None,None,None,None,None,None
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.08,3.0,511,408423,139142,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv...","{'id': 569, 'name': 'MAPPA'}",None,None,None,None,None,None,None,None,None


In [15]:
for col in range(0,10):
    normalized_df3[col] = normalized_df3[col].apply(lambda x: re.sub('[{}\'0-9:,]', '', str(x)))
    normalized_df3[col] = normalized_df3[col].apply(lambda x: x.replace('id', '').replace('name', '').replace('None', '').strip())

In [16]:
# get integer location of the column 0
normalized_df3.columns.get_loc(0)

29

In [17]:
# combine studios
normalized_df3['studios'] = normalized_df3[normalized_df3.columns[29:]].agg(' '.join, axis=1)
normalized_df3['studios'] = normalized_df3['studios'].apply(lambda x: ', '.join([word for word in x.split() if word != '']))

In [18]:
# drop the normalized columns (0 to 9)
normalized_df3.drop([0,1,2,3,4,5,6,7,8,9], axis=1, inplace=True)
normalized_df3.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1.0,3,3149847,2003682,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...",Bones
1,52034,"""Oshi no Ko""",2023-04-12,NaN,Sixteen-year-old Ai Hoshino is a talented and ...,9.08,2.0,525,399851,131985,white,2022-06-09T13:01:38+00:00,2023-04-23T23:12:26+00:00,tv,currently_airing,11,manga,0,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[My Star],[Oshi No Ko],【推しの子】,2023.0,spring,wednesday,23:00,"Drama, Reincarnation, Seinen, Showbiz, Superna...","Doga, Kobo"
2,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.08,3.0,511,408423,139142,white,2022-04-03T15:34:50+00:00,2023-04-12T14:02:06+00:00,special,currently_airing,2,manga,3690,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"[Shingeki no Kyojin: The Final Season Part 3, ...",Attack on Titan: Final Season - The Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv...",MAPPA


In [19]:
normalized_df3.shape

(20000, 30)

In [20]:
normalized_df3.to_csv('data/anime_data.csv', index=False)